In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#特征值
x = tf.placeholder(tf.float32,[None,784],name="X")
#标签值
y = tf.placeholder(tf.float32,[None,10],name="Y")

In [3]:
#隐藏层神经元数量
H1_NN = 256
H2_NN = 64

W1 = tf.Variable(tf.truncated_normal([784,H1_NN],stddev=0.1))
b1 = tf.Variable(tf.zeros([H1_NN]))

W2 = tf.Variable(tf.truncated_normal([H1_NN,H2_NN],stddev=0.1))
b2 = tf.Variable(tf.zeros([H2_NN]))

W3 = tf.Variable(tf.truncated_normal([H2_NN,10],stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

#隐藏层用rulu激活
Y1 = tf.nn.relu(tf.matmul(x,W1)+b1)#第1隐藏层结果 

Y2 = tf.nn.relu(tf.matmul(Y1,W2)+b2)#第2隐藏层结果

#计算输出结果  结果用softmax激活（分类型softmax比较好）
forward = tf.matmul(Y2,W3)+b3
pred = tf.nn.softmax(forward)

TensorFlow提供了结合Softmax的交叉熵损失函数定义方法  
用于避免log(0)的造成的NaN值

In [4]:
#注意：这里代进去的是没有经过SoftMax的forward
loss_function = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits=forward,labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [5]:
#设置参数
train_epochs = 40 #训练40轮
batch_size = 50 #单次训练样本数（批次大小）
total_batch = int(mnist.train.num_examples/batch_size)#一轮训练有多少批次
display_step = 1 #显示粒度
learning_rate = 0.01 #学习率

In [6]:
#这次选用Adam优化器
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

In [7]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [8]:
#模型的保存
save_step=5

import os
ckpt_dir = "./ckpt_dir/"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [9]:
#声明完所有变量后 调用tf.train.Saver()
saver = tf.train.Saver()

#开始训练
from time import time
startTime=time()

sess = tf.Session()
init = tf.global_variables_initializer()#变量初始化
sess.run(init)

for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs,ys = mnist.train.next_batch(batch_size)#读取批次数据
        sess.run(optimizer,feed_dict={x:xs,y:ys})#执行批次训练
        
    #total_batch个批次训练完成后 使用验证数据计算误差与准确率 验证集没有分批
    loss,acc = sess.run([loss_function,accuracy],
                       feed_dict={x:mnist.validation.images,y:mnist.validation.labels})
    
    #打印训练过程中的详细信息
    if (epoch+1) % display_step ==0:
        print("Train Epoch:",'%02d' % (epoch+1),"Loss=","{:.9f}".format(loss),\
              "Accuracy=","{:.4f}".format(acc))
        
    if (epoch+1) % save_step ==0:
        saver.save(sess,os.path.join(ckpt_dir,
                                    'mnist_h256_model_{:06d}.ckpt'.format(epoch+1)))
        print( 'mnist_h256_model_{:06d}.ckpt saved'.format(epoch+1))

saver.save(sess,os.path.join(ckpt_dir,'mnist_h256_model.ckpt'))
print("Model saved!")
duration = time()-startTime
print("Train Finished!")
print("It takes:","{:.2f}".format(duration))

Train Epoch: 01 Loss= 0.159674689 Accuracy= 0.9558
Train Epoch: 02 Loss= 0.143118918 Accuracy= 0.9606
Train Epoch: 03 Loss= 0.149871662 Accuracy= 0.9582
Train Epoch: 04 Loss= 0.142722890 Accuracy= 0.9638
Train Epoch: 05 Loss= 0.118842870 Accuracy= 0.9738
mnist_h256_model_000005.ckpt saved
Train Epoch: 06 Loss= 0.137711257 Accuracy= 0.9694
Train Epoch: 07 Loss= 0.152837500 Accuracy= 0.9682
Train Epoch: 08 Loss= 0.148624286 Accuracy= 0.9726
Train Epoch: 09 Loss= 0.158202693 Accuracy= 0.9734
Train Epoch: 10 Loss= 0.165626585 Accuracy= 0.9730
mnist_h256_model_000010.ckpt saved
Train Epoch: 11 Loss= 0.143554434 Accuracy= 0.9714
Train Epoch: 12 Loss= 0.152044684 Accuracy= 0.9726
Train Epoch: 13 Loss= 0.176515922 Accuracy= 0.9708
Train Epoch: 14 Loss= 0.148503512 Accuracy= 0.9734
Train Epoch: 15 Loss= 0.163351461 Accuracy= 0.9774
mnist_h256_model_000015.ckpt saved
Train Epoch: 16 Loss= 0.178613201 Accuracy= 0.9692
Train Epoch: 17 Loss= 0.200772747 Accuracy= 0.9692
Train Epoch: 18 Loss= 0.2128

In [10]:
#用测试集数据进行测试 看看准确率
acc_test = sess.run(accuracy,
                   feed_dict={x:mnist.test.images,y:mnist.test.labels})
print("Test Accuracy:",acc_test)

Test Accuracy: 0.9755


注：对于隐藏层的构建 可以 写一个函数 不用每次定义W和b